# **Demo Script 2**


This script is the testing portion of the trained model analysis. Specifically this script loads in the test fNIRS and fMRI data, as well as the trained model (from demo 1). The trained model is then run on the testing data and the model performance is found.


In [1]:
import scipy.io as sio
import pickle

import numpy as np
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

## **Test the model on all of Run 2 Data**

In [2]:
## Test a model on each subject in Run 2

# Load fNIRS test data
fnirs_file2 = '../data/fnirs/clip2_noUncertain_zhbo_TxRxS.mat'
test_fNIRS = sio.loadmat(fnirs_file2)['zhbo_TxRxS'].transpose((2, 0, 1))  # reshape from TxRxS to SxTxR
# get shape
print(f'test_fNIRS shape is {test_fNIRS.shape}')

# let's impute missing channels with mean of all other channels
test_fNIRS = np.array(
    [
        np.apply_along_axis(lambda x: np.nan_to_num(x, nan=np.nanmean(x)), axis=1, arr=curr_region) \
            for curr_region in test_fNIRS.transpose((2, 1, 0))  # transpose to shape: RxTxS
    ]
).transpose((2, 1, 0))

# load fMRI test data
fmri_file2 = '../data/fmri/clip2_undenoised_withcartoon_fillmean_TxRxS.mat'
test_fMRI = sio.loadmat(fmri_file2)['bold_TxRxS'].transpose((2, 0, 1))  # reshape from TxRxS to SxTxR
test_fMRI_mean = np.mean(test_fMRI, axis=0)  # already imputed, so we use np.mean instead of np.nanmean
# get shape
print(f'test_fMRI shape is {test_fMRI.shape}')

# Load the model
file_path = '../models/sherlock_run1model.pickle'
with open(file_path, 'rb') as f:
   X_pca, y_pca, lr_reg = pickle.load(f)

# get pc loadings for X and y
X_pca_loadings = X_pca.components_
y_pca_loadings = y_pca.components_

print("Model loaded...")


test_fNIRS shape is (29, 1030, 20)
test_fMRI shape is (17, 1030, 122)
Model loaded...


In [3]:
model_performance = []
model_predicted_tc = []
all_predicted_tc = []

for subj in range(test_fNIRS.shape[0]): # range(1): # range(test_fNIRS.shape[0]):
  curr_test_fNIRS = test_fNIRS[subj,:,:]
  fNIRS_test_pc = X_pca.transform(curr_test_fNIRS)
  model_predicted_PC = lr_reg.predict(X=fNIRS_test_pc)
  model_predicted_tc = y_pca.inverse_transform(model_predicted_PC)
  all_predicted_tc.append(model_predicted_tc)  # Append current prediction to the list
  
all_predicted_tc = np.stack(all_predicted_tc, axis=0)
mean_predicted_tc = np.mean(all_predicted_tc, axis=0)


In [4]:
r_values = np.zeros(test_fMRI_mean.shape[1])

for region in range(test_fMRI_mean.shape[1]):
    # Compute the Pearson correlation for each brain region
    r_values[region], _ = pearsonr(mean_predicted_tc[:, region], test_fMRI_mean[:, region])

median_model_performance = np.median(r_values)

print(median_model_performance)

0.20922782660684555
